# Fitting Thomson Scattering Spectra

This function can be used to fit 1D Thomson spectra. How you collapse experimental data to a nice 1D spectrum is (for now) your problem. This notebook (and the function itself) is still a rough draft.

In [ ]:
%matplotlib inline

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from plasmapy.diagnostics import thomson

To demonstrate the fitting capabilities, we'll first generate some synthetic Thomson data using `thomson.spectral_density` and then adding noise (on the scape of the EPW feature).

In [ ]:
# Generate theoretical spectrum
probe_wavelength = 532*u.nm
wavelengths = np.arange(probe_wavelength.value-60, probe_wavelength.value+60, 0.01)*u.nm
probe_vec = np.array([1, 0, 0])
scattering_angle = np.deg2rad(63)
scatter_vec = np.array([np.cos(scattering_angle), np.sin(scattering_angle), 0])

n = 2e17*u.cm**-3
ion_species = [ 'C-12 5+']
Te = 1e5*u.K
Ti = np.array([50]) * u.eV
electron_vel = np.array([[0, 0, 0]])*u.km/u.s
ion_vel =  np.array([[0, 0, 0]])*u.km/u.s

alpha, Skw_theory = thomson.spectral_density(wavelengths, probe_wavelength,
                     n, Te, Ti, ion_species=ion_species, 
                     electron_vel=electron_vel,ion_vel=ion_vel,
                     probe_vec=probe_vec, scatter_vec=scatter_vec)

# Add noise
noise_level = 1e-14*u.s/u.rad
Skw_noisy = Skw_theory + (np.random.rand(wavelengths.size)-0.5)*noise_level




# PLOTTING

fig, ax = plt.subplots(ncols=2, figsize=(12,4))
fig.subplots_adjust(wspace=0.2)

for a in ax:
    a.set_xlabel("Wavelength (nm)")
    a.set_ylabel("Skw")
ax[0].set_xlim(531, 533)
ax[0].set_title("Ion Acoustic Wave")
ax[0].plot(wavelengths, Skw_noisy)
ax[0].plot(wavelengths, Skw_theory, linewidth=3)

ax[1].set_xlim(520, 545)
ax[1].set_ylim(0, 5e-14)
ax[1].set_title("Electron Plasma Wave")
ax[1].plot(wavelengths, Skw_noisy)
ax[1].plot(wavelengths, Skw_theory, linewidth=3)


Now, we'll use the `thomson.fit_thomson` function to fit this noisey data. First, we create an array of variable names that we want to vary in the fit.

In [ ]:
fit_vars = ['n','Te']

The remainder of the fitting function arguments are exactly the same as the `thomson.spectral_density` arguments, with the addition of `spectral_density` which is the data to be fitted. All inputs will be treated as known quantites except those listed in the `fit_vars` array which will be treated as initial guesses.

In order to illustrate the fitting process, we'll intentionally set incorrect guesses for the density and electron temperaure.

In [ ]:
n_guess = 4e16 * u.cm**-3
Te_guess = 2e5*u.K

result = thomson.fit_thomson(wavelengths, probe_wavelength,
                      Skw_noisy,
                      fit_vars,
                      ion_species = ion_species,
                      probe_vec=probe_vec, scatter_vec=scatter_vec,
                      n = n_guess,
                      Te = Te_guess,
                      Ti = Ti,
                      electron_vel = electron_vel, ion_vel = ion_vel,
                      )

print(f"Te from fit: {result['Te0']}")
print(f"n from fit: {result['n']}")

The results converge...

In [ ]:
Te_fit = result['Te0']*u.K
alpha, Skw_fit = thomson.spectral_density(wavelengths, probe_wavelength,
                     n, Te_fit, Ti, ion_species=ion_species, 
                     electron_vel=electron_vel,ion_vel=ion_vel,
                     probe_vec=probe_vec, scatter_vec=scatter_vec)

fig, ax = plt.subplots(ncols=2, figsize=(12,4))
fig.subplots_adjust(wspace=0.2)

for a in ax:
    a.set_xlabel("Wavelength (nm)")
    a.set_ylabel("Skw")
ax[0].set_xlim(531, 533)
ax[0].set_title("Ion Acoustic Wave")
ax[0].plot(wavelengths, Skw_noisy)
ax[0].plot(wavelengths, Skw_theory, linewidth=3)
ax[0].plot(wavelengths, Skw_fit)

ax[1].set_xlim(520, 545)
ax[1].set_ylim(0, 5e-14)
ax[1].set_title("Electron Plasma Wave")
ax[1].plot(wavelengths, Skw_noisy)
ax[1].plot(wavelengths, Skw_theory, linewidth=3)
ax[1].plot(wavelengths, Skw_fit)

## Fitting Scattering Spectra with Multiple Populations

This function is capable of fitting spectra generated by multiple populations of either ions or electrons. Since the `lmfit` module used for fitting only accepts scalar parameters, the quantites corresponding to different species are split into separate parameters using the naming scheme Te0, Te1 etc.